In [1]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d,highway_conv_2d, avg_pool_2d
from tflearn.layers.normalization import local_response_normalization,batch_normalization
from tflearn.layers.estimator import regression
from tflearn.layers.merge_ops import merge
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
import tflearn.data_utils as du

import tensorflow as tf
import numpy as np
import pickle, random, copy, platform, os, time


def convnet(img_aug=None, drop_out = 0.5):
    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)
    network = conv_2d(Input, 32, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network) #try with batch norm
    network = conv_2d(network, 64, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network)
    network = conv_2d(network, 128, 5, activation='elu', regularizer="L2")
    network = batch_normalization(network)
    network = fully_connected(network, 512, activation='elu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 2048, activation='elu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam',loss='categorical_crossentropy', name='target')
    return network

def highnet(img_aug = None, drop_out = 0.5):
    highnet = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)

    for i in range(3):
        for j in [3, 2, 1]:
            highnet = highway_conv_2d(highnet, 16, j, activation='elu')
        highnet = max_pool_2d(highnet, 2)
        highnet = batch_normalization(highnet)

    highnet = fully_connected(highnet, 128, activation='elu')
    highnet = dropout(highnet,drop_out)
    highnet = fully_connected(highnet, 256, activation='elu')
    highnet = dropout(highnet,drop_out)
    highnet = fully_connected(highnet, 2, activation='softmax')
    highnet = regression(highnet, optimizer='adam', loss='categorical_crossentropy', name='target')
    return highnet

def resnet(img_aug=None, drop_out=None):
    # Building Residual Network
    net = input_data(shape=[None, 75, 75, 2], name='input', data_augmentation=img_aug)
    net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, 5, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, 4, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, 4, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='categorical_crossentropy', name='target')
    return net

def vgg(img_aug=None, drop_out=0.5):    
    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)
    network = conv_2d(Input, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 128, 3, activation='relu')
    network = conv_2d(network, 128, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 2, activation='softmax')
    network = tflearn.regression(network, optimizer='adam',loss='categorical_crossentropy', name='target')
    
    return network

def alexnet(img_aug=None, drop_out=0.5): 
    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)
    network = conv_2d(Input, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, drop_out)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, drop_out)
    network = fully_connected(network, 2, activation='softmax')
    network = tflearn.regression(network, optimizer='adam',loss='categorical_crossentropy',name='target')

    return network

def googlenet(img_aug=None, drop_out=0.4):
    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', name ='input', data_augmentation=img_aug)
    conv1_7_7 = conv_2d(Input, 64, 7, strides=2, activation='relu', name='conv1_7_7_s2')
    pool1_3_3 = max_pool_2d(conv1_7_7, 3, strides=2)
    pool1_3_3 = local_response_normalization(pool1_3_3)
    conv2_3_3_reduce = conv_2d(pool1_3_3, 64, 1, activation='relu', name='conv2_3_3_reduce')
    conv2_3_3 = conv_2d(conv2_3_3_reduce, 192, 3, activation='relu', name='conv2_3_3')
    conv2_3_3 = local_response_normalization(conv2_3_3)
    pool2_3_3 = max_pool_2d(conv2_3_3, kernel_size=3, strides=2, name='pool2_3_3_s2')

    # 3a
    inception_3a_1_1 = conv_2d(pool2_3_3, 64, 1, activation='relu', name='inception_3a_1_1')
    inception_3a_3_3_reduce = conv_2d(pool2_3_3, 96, 1, activation='relu', name='inception_3a_3_3_reduce')
    inception_3a_3_3 = conv_2d(inception_3a_3_3_reduce, 128, filter_size=3,  activation='relu', name='inception_3a_3_3')
    inception_3a_5_5_reduce = conv_2d(pool2_3_3, 16, filter_size=1, activation='relu', name='inception_3a_5_5_reduce')
    inception_3a_5_5 = conv_2d(inception_3a_5_5_reduce, 32, filter_size=5, activation='relu', name='inception_3a_5_5')
    inception_3a_pool = max_pool_2d(pool2_3_3, kernel_size=3, strides=1, name='inception_3a_pool')
    inception_3a_pool_1_1 = conv_2d(inception_3a_pool, 32, filter_size=1, activation='relu', name='inception_3a_pool_1_1')
    inception_3a_output = merge([inception_3a_1_1, inception_3a_3_3, inception_3a_5_5, inception_3a_pool_1_1], mode='concat', axis=3)

    # 3b
    inception_3b_1_1 = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_1_1')
    inception_3b_3_3_reduce = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_3_3_reduce')
    inception_3b_3_3 = conv_2d(inception_3b_3_3_reduce, 192, filter_size=3, activation='relu', name='inception_3b_3_3')
    inception_3b_5_5_reduce = conv_2d(inception_3a_output, 32, filter_size=1, activation='relu', name='inception_3b_5_5_reduce')
    inception_3b_5_5 = conv_2d(inception_3b_5_5_reduce, 96, filter_size=5,  name='inception_3b_5_5')
    inception_3b_pool = max_pool_2d(inception_3a_output, kernel_size=3, strides=1,  name='inception_3b_pool')
    inception_3b_pool_1_1 = conv_2d(inception_3b_pool, 64, filter_size=1, activation='relu', name='inception_3b_pool_1_1')
    inception_3b_output = merge([inception_3b_1_1, inception_3b_3_3, inception_3b_5_5, inception_3b_pool_1_1], mode='concat', axis=3, name='inception_3b_output')
    pool3_3_3 = max_pool_2d(inception_3b_output, kernel_size=3, strides=2, name='pool3_3_3')

    # 4a
    inception_4a_1_1 = conv_2d(pool3_3_3, 192, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4a_3_3_reduce = conv_2d(pool3_3_3, 96, filter_size=1, activation='relu', name='inception_4a_3_3_reduce')
    inception_4a_3_3 = conv_2d(inception_4a_3_3_reduce, 208, filter_size=3,  activation='relu', name='inception_4a_3_3')
    inception_4a_5_5_reduce = conv_2d(pool3_3_3, 16, filter_size=1, activation='relu', name='inception_4a_5_5_reduce')
    inception_4a_5_5 = conv_2d(inception_4a_5_5_reduce, 48, filter_size=5,  activation='relu', name='inception_4a_5_5')
    inception_4a_pool = max_pool_2d(pool3_3_3, kernel_size=3, strides=1,  name='inception_4a_pool')
    inception_4a_pool_1_1 = conv_2d(inception_4a_pool, 64, filter_size=1, activation='relu', name='inception_4a_pool_1_1')
    inception_4a_output = merge([inception_4a_1_1, inception_4a_3_3, inception_4a_5_5, inception_4a_pool_1_1], mode='concat', axis=3, name='inception_4a_output')

    # 4b
    inception_4b_1_1 = conv_2d(inception_4a_output, 160, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4b_3_3_reduce = conv_2d(inception_4a_output, 112, filter_size=1, activation='relu', name='inception_4b_3_3_reduce')
    inception_4b_3_3 = conv_2d(inception_4b_3_3_reduce, 224, filter_size=3, activation='relu', name='inception_4b_3_3')
    inception_4b_5_5_reduce = conv_2d(inception_4a_output, 24, filter_size=1, activation='relu', name='inception_4b_5_5_reduce')
    inception_4b_5_5 = conv_2d(inception_4b_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4b_5_5')
    inception_4b_pool = max_pool_2d(inception_4a_output, kernel_size=3, strides=1,  name='inception_4b_pool')
    inception_4b_pool_1_1 = conv_2d(inception_4b_pool, 64, filter_size=1, activation='relu', name='inception_4b_pool_1_1')
    inception_4b_output = merge([inception_4b_1_1, inception_4b_3_3, inception_4b_5_5, inception_4b_pool_1_1], mode='concat', axis=3, name='inception_4b_output')

    # 4c
    inception_4c_1_1 = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_1_1')
    inception_4c_3_3_reduce = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_3_3_reduce')
    inception_4c_3_3 = conv_2d(inception_4c_3_3_reduce, 256,  filter_size=3, activation='relu', name='inception_4c_3_3')
    inception_4c_5_5_reduce = conv_2d(inception_4b_output, 24, filter_size=1, activation='relu', name='inception_4c_5_5_reduce')
    inception_4c_5_5 = conv_2d(inception_4c_5_5_reduce, 64,  filter_size=5, activation='relu', name='inception_4c_5_5')
    inception_4c_pool = max_pool_2d(inception_4b_output, kernel_size=3, strides=1)
    inception_4c_pool_1_1 = conv_2d(inception_4c_pool, 64, filter_size=1, activation='relu', name='inception_4c_pool_1_1')
    inception_4c_output = merge([inception_4c_1_1, inception_4c_3_3, inception_4c_5_5, inception_4c_pool_1_1], mode='concat', axis=3, name='inception_4c_output')

    # 4d
    inception_4d_1_1 = conv_2d(inception_4c_output, 112, filter_size=1, activation='relu', name='inception_4d_1_1')
    inception_4d_3_3_reduce = conv_2d(inception_4c_output, 144, filter_size=1, activation='relu', name='inception_4d_3_3_reduce')
    inception_4d_3_3 = conv_2d(inception_4d_3_3_reduce, 288, filter_size=3, activation='relu', name='inception_4d_3_3')
    inception_4d_5_5_reduce = conv_2d(inception_4c_output, 32, filter_size=1, activation='relu', name='inception_4d_5_5_reduce')
    inception_4d_5_5 = conv_2d(inception_4d_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4d_5_5')
    inception_4d_pool = max_pool_2d(inception_4c_output, kernel_size=3, strides=1,  name='inception_4d_pool')
    inception_4d_pool_1_1 = conv_2d(inception_4d_pool, 64, filter_size=1, activation='relu', name='inception_4d_pool_1_1')
    inception_4d_output = merge([inception_4d_1_1, inception_4d_3_3, inception_4d_5_5, inception_4d_pool_1_1], mode='concat', axis=3, name='inception_4d_output')

    # 4e
    inception_4e_1_1 = conv_2d(inception_4d_output, 256, filter_size=1, activation='relu', name='inception_4e_1_1')
    inception_4e_3_3_reduce = conv_2d(inception_4d_output, 160, filter_size=1, activation='relu', name='inception_4e_3_3_reduce')
    inception_4e_3_3 = conv_2d(inception_4e_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_4e_3_3')
    inception_4e_5_5_reduce = conv_2d(inception_4d_output, 32, filter_size=1, activation='relu', name='inception_4e_5_5_reduce')
    inception_4e_5_5 = conv_2d(inception_4e_5_5_reduce, 128,  filter_size=5, activation='relu', name='inception_4e_5_5')
    inception_4e_pool = max_pool_2d(inception_4d_output, kernel_size=3, strides=1,  name='inception_4e_pool')
    inception_4e_pool_1_1 = conv_2d(inception_4e_pool, 128, filter_size=1, activation='relu', name='inception_4e_pool_1_1')
    inception_4e_output = merge([inception_4e_1_1, inception_4e_3_3, inception_4e_5_5, inception_4e_pool_1_1], axis=3, mode='concat')
    pool4_3_3 = max_pool_2d(inception_4e_output, kernel_size=3, strides=2, name='pool_3_3')

    # 5a
    inception_5a_1_1 = conv_2d(pool4_3_3, 256, filter_size=1, activation='relu', name='inception_5a_1_1')
    inception_5a_3_3_reduce = conv_2d(pool4_3_3, 160, filter_size=1, activation='relu', name='inception_5a_3_3_reduce')
    inception_5a_3_3 = conv_2d(inception_5a_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_5a_3_3')
    inception_5a_5_5_reduce = conv_2d(pool4_3_3, 32, filter_size=1, activation='relu', name='inception_5a_5_5_reduce')
    inception_5a_5_5 = conv_2d(inception_5a_5_5_reduce, 128, filter_size=5,  activation='relu', name='inception_5a_5_5')
    inception_5a_pool = max_pool_2d(pool4_3_3, kernel_size=3, strides=1,  name='inception_5a_pool')
    inception_5a_pool_1_1 = conv_2d(inception_5a_pool, 128, filter_size=1, activation='relu', name='inception_5a_pool_1_1')
    inception_5a_output = merge([inception_5a_1_1, inception_5a_3_3, inception_5a_5_5, inception_5a_pool_1_1], axis=3, mode='concat')

    # 5b
    inception_5b_1_1 = conv_2d(inception_5a_output, 384, filter_size=1, activation='relu', name='inception_5b_1_1')
    inception_5b_3_3_reduce = conv_2d(inception_5a_output, 192, filter_size=1, activation='relu', name='inception_5b_3_3_reduce')
    inception_5b_3_3 = conv_2d(inception_5b_3_3_reduce, 384,  filter_size=3, activation='relu', name='inception_5b_3_3')
    inception_5b_5_5_reduce = conv_2d(inception_5a_output, 48, filter_size=1, activation='relu', name='inception_5b_5_5_reduce')
    inception_5b_5_5 = conv_2d(inception_5b_5_5_reduce, 128, filter_size=5, activation='relu', name='inception_5b_5_5')
    inception_5b_pool = max_pool_2d(inception_5a_output, kernel_size=3, strides=1,  name='inception_5b_pool')
    inception_5b_pool_1_1 = conv_2d(inception_5b_pool, 128, filter_size=1, activation='relu', name='inception_5b_pool_1_1')
    inception_5b_output = merge([inception_5b_1_1, inception_5b_3_3, inception_5b_5_5, inception_5b_pool_1_1], axis=3, mode='concat')
    pool5_7_7 = avg_pool_2d(inception_5b_output, kernel_size=7, strides=1)
    pool5_7_7 = dropout(pool5_7_7, drop_out)

    # fc
    net = fully_connected(pool5_7_7, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='categorical_crossentropy',name='target')
    
    return net

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
def get_save_path(net_name):
    return save_dir + str(net_name)

In [3]:
class EarlyStoppingCallback(tflearn.callbacks.Callback):
    def __init__(self, val_acc_thresh):
        """ Note: We are free to define our init function however we please. """
        self.val_acc_thresh = val_acc_thresh
    
    def on_epoch_end(self, training_state):
        """ """
        # Apparently this can happen.
        if training_state.val_acc is None: return
        if training_state.val_acc > self.val_acc_thresh:
            raise StopIteration

In [4]:
def predict(model, test_data):
    #8424=8*81*13
    batch_size =104
    iteration = int(len(test_data)/batch_size)  #iter=81  
    prediction = []
    
    for i in range(iteration):
        pred = model.predict(test_data[i*batch_size: (i+1)*batch_size])
        prediction.extend(pred.flatten())
    prediction = np.reshape(prediction, (-1,2))
    
    return prediction


def predict_result(model, test_data, ref, mode=None):
    #8424=8*81*13
    #hypotheses = np.zeros((Xte.shape[0],ensemble,2))
    batch_size =104
    iteration = int(len(test_data)/batch_size)  #iter=81  
    with open("C:/Users/User/Desktop/kaggle/result{0}.txt".format(mode), "w") as f:
        f.write("id,is_iceberg\n")
        for i in range(iteration):
            pred = model.predict(test_data[i*batch_size: (i+1)*batch_size])
            for j in range(batch_size):
                data = "%s,%f\n" % (ref[i*batch_size+j], pred[j][1])
                f.write(data)



In [5]:
def onehot_encoding (Y, y_class):
    Y_onehot = np.zeros((Y.size, int(y_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

with open('resnet_data', 'rb') as f:
    data = pickle.load(f)
    Xtr = data[0]
    Xtr = np.reshape(Xtr, (-1,75,75,2))
    Ytr = data[1]
    Xte =data[2]
    Xte = np.reshape(Xte, (-1,75,75,2))
    rf = data[3]

Ytr = onehot_encoding(Ytr, 2) 
Xtr, Ytr = tflearn.data_utils.shuffle(Xtr, Ytr)

X, Y = Xtr[:1200], Ytr[:1200]
validX, validY =Xtr[1200:1500], Ytr[1200:1500]
testX, testY = Xtr[1500:],Ytr[1500:]

X, mean = du.featurewise_zero_center(X)
testX = du.featurewise_zero_center(testX, mean)
validX = du.featurewise_zero_center(validX, mean)

#parameters
n_epoch=40
batch_size = 100
ensemble=8   #Total no of ensemble models
n_ensemble=0   #n_semble counter

hypotheses_test = np.zeros((ensemble,testX.shape[0],2))
hypotheses_valid = np.zeros((ensemble,validY.shape[0],2))
hypotheses = np.zeros((ensemble,Xte.shape[0],2))

early_stopping_cb = EarlyStoppingCallback(val_acc_thresh=0.95)
#img_prep.add_custom_preprocessing (func) <-함수 적용가능: 추가사항

img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_90degrees_rotation(rotations=[0, 1, 2, 3])
img_aug.add_random_crop([75,75,2], padding=4)

In [6]:
# Training graphs
convnet1 = tf.Graph()
convnet2 = tf.Graph()
highnet1 = tf.Graph()
highnet2 = tf.Graph()
resnet1 = tf.Graph()
resnet2 = tf.Graph()
vgg1 = tf.Graph()
vgg2 = tf.Graph()
alexnet1 = tf.Graph()
alexnet2 = tf.Graph()
googlenet1 = tf.Graph()
googlenet2 = tf.Graph()

#ensemble trial
'''
with convnet1.as_default():
    tf.set_random_seed(777) #no reproducibility...T.T
    tflearn.init_graph(seed=777)
    network1 = convnet(img_aug=img_aug, drop_out=0.7)
    model = tflearn.DNN(network1,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model.fit({'input': X}, {'target': Y}, n_epoch=n_epoch, callbacks=early_stopping_cb,batch_size=batch_size,
               validation_set=({'input': validX}, {'target': validY}),show_metric=True, run_id='conv1')
    model.save('convnet1')
    hypotheses[n_ensemble]=predict(model, Xte)
    hypotheses_test[n_ensemble]=model.predict(testX)
    hypotheses_valid[n_ensemble]=model.predict(validX)
    predict_result(model, Xte, rf, mode='convet1')
    n_ensemble+=1
tf.reset_default_graph()

with convnet2.as_default():
    tf.set_random_seed(777) 
    tflearn.init_graph(seed=777)
    network2 = convnet(img_aug=img_aug, drop_out=0.6)
    model2 = tflearn.DNN(network2, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model2.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,callbacks=early_stopping_cb,batch_size=batch_size,
               validation_set=({'input': validX}, {'target': validY}),show_metric=True, run_id='conv2')
    model2.save('convnet2')
    hypotheses[n_ensemble]=predict(model2, Xte)
    hypotheses_test[n_ensemble]=model2.predict(testX)
    hypotheses_valid[n_ensemble]=model2.predict(validX)
    predict_result(model2, Xte, rf, mode='convet2')
    n_ensemble+=1
tf.reset_default_graph()

with highnet1.as_default():
    tf.set_random_seed(779)
    tflearn.init_graph(seed=779)
    network3 = highnet(img_aug=img_aug,drop_out=0.6)
    model3 = tflearn.DNN(network3, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model3.fit({'input': X}, {'target': Y}, n_epoch=n_epoch, validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='high1', batch_size=batch_size, callbacks=early_stopping_cb)
    model3.save('high1')
    hypotheses[n_ensemble]=predict(model3, Xte)
    hypotheses_test[n_ensemble]=model3.predict(testX)
    hypotheses_valid[n_ensemble]=model3.predict(validX)
    predict_result(model3, Xte, rf, mode='highnet1')
    n_ensemble+=1

tf.reset_default_graph()
with highnet2.as_default():
    tf.set_random_seed(780)
    tflearn.init_graph(seed=780)
    network4 = highnet(img_aug=img_aug,drop_out=0.5)
    model4 = tflearn.DNN(network4, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model4.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='high2', batch_size=batch_size, callbacks=early_stopping_cb)
    model4.save('high2')
    hypotheses[n_ensemble]=predict(model4, Xte)
    hypotheses_test[n_ensemble]=model4.predict(testX)
    hypotheses_valid[n_ensemble]=model4.predict(validX)
    predict_result(model4, Xte, rf, mode='highnet2')
    n_ensemble+=1
tf.reset_default_graph()
'''
with resnet1.as_default():
    tflearn.init_graph()

    network5 = resnet(img_aug=img_aug,drop_out=0.7)
    model5 = tflearn.DNN(network5, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')

    model5.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='resnet1', batch_size=batch_size, callbacks=early_stopping_cb)
    model5.save('resnet1')
    hypotheses[n_ensemble]=predict(model5, Xte)
    hypotheses_test[n_ensemble]=model5.predict(testX)
    hypotheses_valid[n_ensemble]=model5.predict(validX)
    predict_result(model5, Xte, rf, mode='resnet1')
    n_ensemble+=1
tf.reset_default_graph()

with resnet2.as_default():
    tflearn.init_graph()

    network6 = resnet(img_aug=img_aug,drop_out=0.6)
    model6 = tflearn.DNN(network6, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model6.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='resnet2',batch_size=batch_size, callbacks=early_stopping_cb)
    model6.save('resnet2')
    hypotheses[n_ensemble]=predict(model6, Xte)
    hypotheses_test[n_ensemble]=model6.predict(testX)
    hypotheses_valid[n_ensemble]=model6.predict(validX)
    predict_result(model6, Xte, rf, mode='resnet2')
    n_ensemble+=1
tf.reset_default_graph()

with vgg1.as_default():
    tflearn.init_graph()

    network7 = vgg(img_aug=img_aug,drop_out=0.6)
    model7 = tflearn.DNN(network7, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model7.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='vgg1',batch_size=batch_size, callbacks=early_stopping_cb)
    model7.save('vgg1')
    hypotheses[n_ensemble]=predict(model7, Xte)
    hypotheses_test[n_ensemble]=model7.predict(testX)
    hypotheses_valid[n_ensemble]=model7.predict(validX)
    predict_result(model7, Xte, rf, mode='vgg1')
    n_ensemble+=1
tf.reset_default_graph()

with vgg2.as_default():
    tflearn.init_graph()

    network8 = vgg(img_aug=img_aug,drop_out=0.7)
    model8= tflearn.DNN(network8, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model8.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='vgg2',batch_size=batch_size, callbacks=early_stopping_cb)
    model8.save('vgg2')
    hypotheses[n_ensemble]=predict(model8, Xte)
    hypotheses_test[n_ensemble]=model8.predict(testX)
    hypotheses_valid[n_ensemble]=model8.predict(validX)
    predict_result(model8, Xte, rf, mode='vgg2')
    n_ensemble+=1
tf.reset_default_graph()

with alexnet1.as_default():
    tflearn.init_graph()

    network9= alexnet(img_aug=img_aug, drop_out=0.7)
    model9 = tflearn.DNN(network9, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model9.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='alex1',batch_size=batch_size, callbacks=early_stopping_cb)
    model9.save('alex1')
    hypotheses[n_ensemble]=predict(model9, Xte)
    hypotheses_test[n_ensemble]=model9.predict(testX)
    hypotheses_valid[n_ensemble]=model9.predict(validX)
    predict_result(model9, Xte, rf, mode='alexnet1')
    n_ensemble+=1
tf.reset_default_graph()

with alexnet2.as_default():
    tflearn.init_graph()

    network10= alexnet(img_aug=img_aug,drop_out=0.6)
    model10 = tflearn.DNN(network10,tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model10.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
                show_metric=True, run_id='alex2',batch_size=batch_size, callbacks=early_stopping_cb)
    model10.save('alex2')
    hypotheses[n_ensemble]=predict(model10, Xte)
    hypotheses_test[n_ensemble]=model10.predict(testX)
    hypotheses_valid[n_ensemble]=model10.predict(validX)
    predict_result(model10, Xte, rf, mode='alexnet2')
    n_ensemble+=1
tf.reset_default_graph()

with googlenet1.as_default():
    tflearn.init_graph()

    network11= googlenet(img_aug=img_aug, drop_out=0.4)
    model11 = tflearn.DNN(network11, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model11.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='googlenet1',batch_size=batch_size, callbacks=early_stopping_cb)
    model11.save('googlenet1')
    hypotheses[n_ensemble]= predict(model11, Xte)
    hypotheses_test[n_ensemble]=model11.predict(testX)
    hypotheses_valid[n_ensemble]=model11.predict(validX)
    predict_result(model11, Xte, rf, mode='googlenet1')
    n_ensemble+=1
tf.reset_default_graph()

with googlenet2.as_default():
    tflearn.init_graph()

    network12= googlenet(img_aug=img_aug, drop_out=0.5)
    model12 = tflearn.DNN(network12, tensorboard_verbose=3,tensorboard_dir='c:\\tflearn_logs\\')
    model12.fit({'input': X}, {'target': Y}, n_epoch=n_epoch,validation_set=({'input': validX}, {'target': validY}),
               show_metric=True, run_id='googlenet2',batch_size=batch_size, callbacks=early_stopping_cb)
    model12.save('googlenet2')
    hypotheses[n_ensemble]= predict(model12, Xte)
    hypotheses_test[n_ensemble]=model12.predict(testX)
    hypotheses_valid[n_ensemble]=model12.predict(validX)
    predict_result(model12, Xte, rf, mode='googlenet2')
    n_ensemble+=1
tf.reset_default_graph()

#End of training

print("Optimization finished")

Training Step: 479  | total loss: 10.91256 | time: 90.031s
| Adam | epoch: 040 | loss: 10.91256 - acc: 0.5261 -- iter: 1100/1200
Training Step: 480  | total loss: 10.88050 | time: 99.299s
| Adam | epoch: 040 | loss: 10.88050 - acc: 0.5275 | val_loss: 9.97787 - val_acc: 0.5667 -- iter: 1200/1200
--
INFO:tensorflow:C:\Users\User\Desktop\kaggle\vgg2 is not in all_model_checkpoint_paths. Manually adding it.


ResourceExhaustedError: OOM when allocating tensor with shape[300,64,75,75]
	 [[Node: Conv2D_1/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Conv2D/Relu, Conv2D_1/W/read)]]
	 [[Node: FullyConnected_2/BiasAdd/_641 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_161_FullyConnected_2/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Conv2D_1/Conv2D', defined at:
  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_l

In [ ]:
#Ensemble weight training : using Validation set
hypotheses = hypotheses.transpose([1,0,2])
hypotheses_valid = hypotheses_valid.transpose([1,0,2])
hypotheses_test = hypotheses_test.transpose([1,0,2])
hypothesis = np.zeros((testX.shape[0],2))
result = np.zeros((Xte.shape[0],2))

ens=tf.Graph()
with ens.as_default():
    ensemble_net = input_data(shape=[None, n_ensemble, 2], name='input')
    ensemble_net = fully_connected(ensemble_net, 2, activation='softplus')
    ensemble_net = regression(ensemble_net, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    ens_model = tflearn.DNN(ensemble_net, tensorboard_verbose=0)
    ens_model.fit({'input': hypotheses_valid}, {'target': validY}, n_epoch=20, show_metric=False, run_id='ensemble',batch_size=100)
    hypothesis = ens_model.predict(hypotheses_test)

    predict_result(ens_model, hypotheses, rf, mode='ensemble')
    
    print(hypothesis, hypothesis.shape)
    #predict(ens_model, Xte, rf, ensemble)
    ens_model.save('ens_model')

correct_prediction = np.equal(np.argmax(hypothesis,1), np.argmax(testY, 1))
accuracy_ens = np.mean(correct_prediction)

print("Final Accuracy :", accuracy_ens)

In [ ]:
print(np.argmax(hypothesis,1))

In [ ]:
print(np.argmax(testY, 1))

In [2]:
12*16

192

# Retrieve variable responsible for managing training mode
training_mode = tflearn.get_training_mode()

# Define a conditional op
my_conditional_op = tf.cond(training_mode, if_yes_op, if_no_op)
# Set training mode to True
tflearn.is_training(True)
session.run(my_conditional_op)
if_yes_op
 # Set training mode to False
tflearn.is_training(False)
session.run(my_conditio

z = tf.multiply(a, b)
result = tf.cond(x < y, lambda: tf.add(x, z), lambda: tf.square(y))